In [89]:
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import logit
import pandas as pd
from scipy.stats import shapiro, levene
import seaborn as sns
import matplotlib.pyplot as plt
from typing import Dict, Tuple, Any

In [90]:
def warn(*args, **kwargs):
     pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

In [91]:
ownership = pd.read_csv("../../data/final_data/final_ownership_data.csv")
sales = pd.read_csv("../../data/final_data/final_sales_data.csv")

In [92]:
ownership.columns

Index(['Unnamed: 0', '_c0', 'Occupation', 'Annual_Income', 'Credit_Score',
       'Years_of_Employment', 'Finance_Status', 'Car', 'Number_of_Children'],
      dtype='object')

In [93]:
sales.columns

Index(['Unnamed: 0', '_c0', 'Car_id', 'Date', 'Gender', 'Annual_Income',
       'Dealer_Name', 'Company', 'Model', 'Engine', 'Transmission', 'Color',
       'Price', 'Dealer_No ', 'Body_Style', 'Phone', 'Dealer_Region', 'Month',
       'Year'],
      dtype='object')

In [94]:

def t_test(df, column1, column2, alpha=0.05, plot=False):

     try:
          # Input validation
          if not isinstance(df, pd.DataFrame):
               raise TypeError("Input must be a pandas DataFrame")
          if not all(col in df.columns for col in [column1, column2]):
               raise ValueError("Specified columns not found in DataFrame")
     
          # Separate groups
          group1 = df[df[column1] == True][column2].dropna()
          group2 = df[df[column1] == False][column2].dropna()
     
          if len(group1) == 0 or len(group2) == 0:
               raise ValueError("One or both groups are empty after removing NaN values")
     
          # Descriptive statistics
          stats_df = pd.DataFrame({
               'n': [len(group1), len(group2)],
               'mean': [group1.mean(), group2.mean()],
               'std': [group1.std(), group2.std()],
               'se': [group1.std()/np.sqrt(len(group1)),
                      group2.std()/np.sqrt(len(group2))]
          }, index=['Group 1', 'Group 2'])
     
          # Assumption checks
          normality_1 = shapiro(group1) if len(group1) < 5000 else (1, 1)
          normality_2 = shapiro(group2) if len(group2) < 5000 else (1, 1)
          variance_test = levene(group1, group2)
     
          # Perform t-test
          t_stat, p_value = stats.ttest_ind(group1, group2)
     
          # Effect size (Cohen's d)
          pooled_std = np.sqrt(((len(group1) - 1) * group1.var() +
                                (len(group2) - 1) * group2.var()) /
                               (len(group1) + len(group2) - 2))
          cohens_d = abs(group1.mean() - group2.mean()) / pooled_std
     
          # Optional plotting
          if plot:
               plt.figure(figsize=(10, 6))
               sns.boxplot(x=column1, y=column2, data=df)
               plt.title(f'{column2} by {column1}')
               plt.show()
     
          # Print results
          print("\nIndependent t-test Results")
          print("-" * 50)
          print("\nDescriptive Statistics:")
          print(stats_df)
          print("\nAssumption Checks:")
          print(f"Normality test Group 1: p = {normality_1[1]:.4f}")
          print(f"Normality test Group 2: p = {normality_2[1]:.4f}")
          print(f"Levene's test for equal variances: p = {variance_test[1]:.4f}")
          print("\nTest Results:")
          print(f"T-statistic: {t_stat:.4f}")
          print(f"P-value: {p_value:.4f}")
          print(f"Cohen's d: {cohens_d:.4f}")
     
          return {
               'statistics': stats_df,
               't_stat': t_stat,
               'p_value': p_value,
               'cohens_d': cohens_d,
               'normality_tests': (normality_1, normality_2),
               'variance_test': variance_test
          }
     
     except Exception as e:
          print(f"Error occurred: {str(e)}")
          return None


def one_way_anova(df: pd.DataFrame, numerical_column: str, categorical_column: str, alpha: float = 0.05, plot: bool = False):

     try:
          # Input validation
          if not isinstance(df, pd.DataFrame):
               raise TypeError("Input must be a pandas DataFrame")
          if not all(col in df.columns for col in [numerical_column, categorical_column]):
               raise ValueError("Specified columns not found in DataFrame")

          # Create groups and remove NaN values
          groups = [group[numerical_column].dropna().values
                    for name, group in df.groupby(categorical_column)]
          group_names = df[categorical_column].unique()

          if any(len(group) == 0 for group in groups):
               raise ValueError("One or more groups are empty after removing NaN values")

          # Descriptive statistics
          stats_df = df.groupby(categorical_column)[numerical_column].agg([
               ('count', 'count'),
               ('mean', 'mean'),
               ('std', 'std'),
               ('min', 'min'),
               ('max', 'max')
          ]).round(2)

          # Assumption checks
          assumptions = {}

          # 1. Normality test for each group
          assumptions['normality'] = {}
          for name, group in zip(group_names, groups):
               if len(group) < 5000:  # Shapiro-Wilk test for smaller samples
                    stat, p_value = shapiro(group)
                    assumptions['normality'][name] = {'statistic': stat, 'p_value': p_value}

          # 2. Homogeneity of variances
          assumptions['levene'] = levene(*groups)

          # Perform one-way ANOVA
          f_stat, p_value = stats.f_oneway(*groups)

          # Calculate effect size (eta-squared)
          df_total = len(df[numerical_column]) - 1
          df_between = len(groups) - 1
          df_within = df_total - df_between

          ss_between = sum(len(group) * ((group.mean() - df[numerical_column].mean())**2)
                           for group in groups)
          ss_total = sum((x - df[numerical_column].mean())**2
                         for x in df[numerical_column])

          eta_squared = ss_between / ss_total

          # Perform Tukey's HSD test if ANOVA is significant
          if p_value < alpha:
               tukey = pairwise_tukeyhsd(df[numerical_column], df[categorical_column])
               tukey_results = pd.DataFrame(data=tukey._results_table.data[1:],
                                            columns=tukey._results_table.data[0])
          else:
               tukey_results = None

          # Optional plotting
          if plot:
               fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

               # Boxplot
               sns.boxplot(x=categorical_column, y=numerical_column, data=df, ax=ax1)
               ax1.set_title('Distribution by Group')

               # QQ plot for each group
               for name, group in zip(group_names, groups):
                    stats.probplot(group, dist="norm", plot=ax2)
               ax2.set_title('Q-Q Plots by Group')

               plt.tight_layout()
               plt.show()

          # Print results
          print("\nOne-way ANOVA Results")
          print("-" * 50)
          print("\nDescriptive Statistics:")
          print(stats_df)

          print("\nAssumption Checks:")
          for name, results in assumptions['normality'].items():
               print(f"Normality test for {name}: p = {results['p_value']:.4f}")
          print(f"Levene's test for equal variances: p = {assumptions['levene'][1]:.4f}")

          print("\nANOVA Results:")
          print(f"F-statistic: {f_stat:.4f}")
          print(f"P-value: {p_value:.4f}")
          print(f"Eta-squared: {eta_squared:.4f}")

          if tukey_results is not None:
               print("\nTukey's HSD Results:")
               print(tukey_results)

          return {
               'statistics': stats_df,
               'assumptions': assumptions,
               'f_statistic': f_stat,
               'p_value': p_value,
               'eta_squared': eta_squared,
               'tukey_results': tukey_results
          }

     except Exception as e:
          print(f"Error occurred: {str(e)}")
          return None

def chi_square_test(df: pd.DataFrame, column1: str, column2: str, alpha: float = 0.05, plot: bool = False):

     try:
          # Input validation
          if not isinstance(df, pd.DataFrame):
               raise TypeError("Input must be a pandas DataFrame")
          if not all(col in df.columns for col in [column1, column2]):
               raise ValueError("Specified columns not found in DataFrame")

          # Create contingency table
          contingency = pd.crosstab(df[column1], df[column2])

          # Check minimum expected frequencies
          chi2, p_value, dof, expected = stats.chi2_contingency(contingency)
          min_expected = np.min(expected)

          if min_expected < 5:
               warnings.warn("Warning: Some expected frequencies are < 5, "
                             "chi-square test may not be valid")

          # Calculate Cramer's V effect size
          n = contingency.sum().sum()
          min_dim = min(contingency.shape) - 1
          cramer_v = np.sqrt(chi2 / (n * min_dim))

          # Calculate standardized residuals
          observed = contingency.values
          standardized_residuals = (observed - expected) / np.sqrt(expected)
          residuals_df = pd.DataFrame(standardized_residuals,
                                      index=contingency.index,
                                      columns=contingency.columns)

          # Calculate percentages
          row_percentages = contingency.div(contingency.sum(axis=1), axis=0) * 100
          col_percentages = contingency.div(contingency.sum(axis=0), axis=1) * 100

          # Optional plotting
          if plot:
               plt.figure(figsize=(10, 6))
               sns.heatmap(contingency, annot=True, fmt='d', cmap='YlGnBu')
               plt.title('Contingency Table Heatmap')
               plt.show()

          # Print results
          print("\nChi-square Test Results")
          print("-" * 50)
          print("\nContingency Table (Counts):")
          print(contingency)

          print("\nRow Percentages:")
          print(row_percentages.round(2))

          print("\nColumn Percentages:")
          print(col_percentages.round(2))

          print("\nTest Results:")
          print(f"Chi-square statistic: {chi2:.4f}")
          print(f"P-value: {p_value:.4f}")
          print(f"Degrees of freedom: {dof}")
          print(f"Minimum expected frequency: {min_expected:.4f}")
          print(f"Cramer's V: {cramer_v:.4f}")

          print("\nStandardized Residuals:")
          print(residuals_df.round(2))

          return {
               'contingency_table': contingency,
               'row_percentages': row_percentages,
               'column_percentages': col_percentages,
               'chi_square': chi2,
               'p_value': p_value,
               'dof': dof,
               'min_expected': min_expected,
               'cramers_v': cramer_v,
               'standardized_residuals': residuals_df
          }

     except Exception as e:
          print(f"Error occurred: {str(e)}")
          return None

def mann_whitney_test(df, numeric_column, binary_column):
     """
     Performs Mann-Whitney U test between two groups
     """
     group1 = df[df[binary_column] == True][numeric_column]
     group2 = df[df[binary_column] == False][numeric_column]

     # Perform test
     stat, p_value = stats.mannwhitneyu(group1, group2, alternative='two-sided')

     print(f"Mann-Whitney U statistic: {stat:.4f}")
     print(f"p-value: {p_value:.4f}")
     return stat, p_value

def multiple_linear_regression(df, dependent_var, independent_vars):
     """
     Performs multiple linear regression
     """
     # Prepare the data
     X = df[independent_vars]
     # Add constant for intercept
     X = sm.add_constant(X)
     y = df[dependent_var]

     # Fit the model
     model = sm.OLS(y, X).fit()

     print(model.summary())
     return model

def time_series_analysis(df, date_column, value_column):
     """
     Performs Augmented Dickey-Fuller test for stationarity
     """
     # Ensure data is sorted by date
     df = df.sort_values(date_column)

     # Perform ADF test
     result = adfuller(df[value_column].values)

     print('ADF Statistic:', result[0])
     print('p-value:', result[1])
     print('Critical values:')
     for key, value in result[4].items():
          print('\t%s: %.3f' % (key, value))
     return result


In [95]:
result_occupation_car = chi_square_test(ownership, 'Occupation', 'Car')


Chi-square Test Results
--------------------------------------------------

Contingency Table (Counts):
Car                      No  Yes
Occupation                      
Account Executive         0    3
Account Manager           0    2
Accountant                1    8
Architect                 1   10
Art Director              0    1
...                      ..  ...
Veterinarian Technician   1    0
Waiter/Waitress           1    0
Web Designer              4    3
Web Developer             1    6
Writer                    7    0

[116 rows x 2 columns]

Row Percentages:
Car                          No     Yes
Occupation                             
Account Executive          0.00  100.00
Account Manager            0.00  100.00
Accountant                11.11   88.89
Architect                  9.09   90.91
Art Director               0.00  100.00
...                         ...     ...
Veterinarian Technician  100.00    0.00
Waiter/Waitress          100.00    0.00
Web Designer            

In [96]:
result_finance_car = chi_square_test(ownership, 'Finance_Status', 'Car')


Chi-square Test Results
--------------------------------------------------

Contingency Table (Counts):
Car             No  Yes
Finance_Status         
Excellent        3    8
Fair            10    4
Good             3   10
Poor             4    0
Stable          49  220
Unknown         10    3
Unstable        67    5

Row Percentages:
Car                 No    Yes
Finance_Status               
Excellent        27.27  72.73
Fair             71.43  28.57
Good             23.08  76.92
Poor            100.00   0.00
Stable           18.22  81.78
Unknown          76.92  23.08
Unstable         93.06   6.94

Column Percentages:
Car                No   Yes
Finance_Status             
Excellent        2.05   3.2
Fair             6.85   1.6
Good             2.05   4.0
Poor             2.74   0.0
Stable          33.56  88.0
Unknown          6.85   1.2
Unstable        45.89   2.0

Test Results:
Chi-square statistic: 162.3593
P-value: 0.0000
Degrees of freedom: 6
Minimum expected frequency: 1.4747

In [97]:
ownership_tt = ownership.__deepcopy__()
ownership_tt['Car']= ownership_tt['Car'].replace({'Yes': 1, 'No': 0}).astype(int)

In [98]:
result_car_employment = t_test(ownership_tt, 'Car', 'Years_of_Employment')


Independent t-test Results
--------------------------------------------------

Descriptive Statistics:
           n      mean       std        se
Group 1  250  5.596000  2.222734  0.140578
Group 2  146  2.376712  1.198507  0.099189

Assumption Checks:
Normality test Group 1: p = 0.0000
Normality test Group 2: p = 0.0000
Levene's test for equal variances: p = 0.0000

Test Results:
T-statistic: 16.1754
P-value: 0.0000
Cohen's d: 1.6848


In [99]:
result_car_children = chi_square_test(ownership, 'Car', 'Number_of_Children')


Chi-square Test Results
--------------------------------------------------

Contingency Table (Counts):
Number_of_Children  -1   0   1   2   3   4
Car                                       
No                  42  38  32  25   8   1
Yes                 37  74  68  62   9   0

Row Percentages:
Number_of_Children     -1      0      1      2     3     4
Car                                                       
No                  28.77  26.03  21.92  17.12  5.48  0.68
Yes                 14.80  29.60  27.20  24.80  3.60  0.00

Column Percentages:
Number_of_Children     -1      0     1      2      3      4
Car                                                        
No                  53.16  33.93  32.0  28.74  47.06  100.0
Yes                 46.84  66.07  68.0  71.26  52.94    0.0

Test Results:
Chi-square statistic: 15.3907
P-value: 0.0088
Degrees of freedom: 5
Minimum expected frequency: 0.3687
Cramer's V: 0.1971

Standardized Residuals:
Number_of_Children    -1     0     1     2    

In [100]:
result_income_car = mann_whitney_test(ownership_tt, 'Annual_Income', 'Car')

Mann-Whitney U statistic: 32367.5000
p-value: 0.0000


In [101]:
sales_mlr = sales.__deepcopy__()
sales_mlr['Gender'] = sales_mlr['Gender'].replace({'Male': 1, 'Female': 0}).astype(int)

In [102]:
result_gender_income_price = multiple_linear_regression(sales_mlr, 'Gender', ['Annual_Income', 'Price'])

                            OLS Regression Results                            
Dep. Variable:                 Gender   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     35.85
Date:                Fri, 29 Nov 2024   Prob (F-statistic):           2.84e-16
Time:                        13:47:18   Log-Likelihood:                -12565.
No. Observations:               23906   AIC:                         2.514e+04
Df Residuals:                   23903   BIC:                         2.516e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.7663      0.006    119.298

In [103]:
result_gender_dealer = chi_square_test(sales, 'Gender', 'Dealer_Name')


Chi-square Test Results
--------------------------------------------------

Contingency Table (Counts):
Dealer_Name  Buddy Storbeck's Diesel Service Inc  C & M Motors Inc  \
Gender                                                               
Female                                       118               133   
Male                                         509               492   

Dealer_Name  Capitol KIA  Chrysler Plymouth  Chrysler of Tri-Cities  \
Gender                                                                
Female               140                151                     136   
Male                 488                474                     490   

Dealer_Name  Classic Chevy  Clay Johnson Auto Sales  Diehl Motor CO Inc  \
Gender                                                                    
Female                 129                      136                 138   
Male                   494                      491                 486   

Dealer_Name  Enterprise Rent

In [104]:
result_date_price = time_series_analysis(sales, 'Date', 'Price')

ADF Statistic: -108.8116803559421
p-value: 0.0
Critical values:
	1%: -3.431
	5%: -2.862
	10%: -2.567


In [105]:
result_price_dealer = one_way_anova(sales, 'Price', 'Dealer_Name')


One-way ANOVA Results
--------------------------------------------------

Descriptive Statistics:
                                                 count      mean       std  \
Dealer_Name                                                                  
Buddy Storbeck's Diesel Service Inc                627  27217.26  14136.72   
C & M Motors Inc                                   625  28111.76  14305.50   
Capitol KIA                                        628  28189.70  15037.23   
Chrysler Plymouth                                  625  27555.53  14478.30   
Chrysler of Tri-Cities                             626  28123.09  14916.47   
Classic Chevy                                      623  28602.01  15310.96   
Clay Johnson Auto Sales                            627  27816.03  14289.07   
Diehl Motor CO Inc                                 624  27993.93  14246.90   
Enterprise Rent A Car                              625  28312.58  14983.85   
Gartner Buick Hyundai Saab                 

In [106]:
result_gender_style = chi_square_test(sales, 'Gender', 'Body_Style')


Chi-square Test Results
--------------------------------------------------

Contingency Table (Counts):
Body_Style  Hardtop  Hatchback  Passenger   SUV  Sedan
Gender                                                
Female          647       1298        883  1335    945
Male           2324       4830       3062  5039   3543

Row Percentages:
Body_Style  Hardtop  Hatchback  Passenger    SUV  Sedan
Gender                                                 
Female        12.67      25.41      17.29  26.14  18.50
Male          12.36      25.69      16.29  26.81  18.85

Column Percentages:
Body_Style  Hardtop  Hatchback  Passenger    SUV  Sedan
Gender                                                 
Female        21.78      21.18      22.38  20.94  21.06
Male          78.22      78.82      77.62  79.06  78.94

Test Results:
Chi-square statistic: 3.7811
P-value: 0.4364
Degrees of freedom: 4
Minimum expected frequency: 634.8142
Cramer's V: 0.0126

Standardized Residuals:
Body_Style  Hardtop  Hatc

In [107]:
result_price_region = one_way_anova(sales, 'Price', 'Dealer_Region')


One-way ANOVA Results
--------------------------------------------------

Descriptive Statistics:
                count      mean       std   min    max
Dealer_Region                                         
Arizona          3433  27954.96  14902.92  1450  85001
Colorado         3130  28334.63  15026.21  9000  85800
Connecticut      3128  27856.34  14619.84  1700  85300
South Carolina   3128  28180.82  15101.54  1200  85200
Texas            4135  28341.60  14903.88  9000  85601
Washington       3131  28119.04  14659.32  9000  85600
Wisconsin        3821  27833.35  14345.00  4300  85400

Assumption Checks:
Normality test for Connecticut: p = 0.0000
Normality test for Colorado: p = 0.0000
Normality test for South Carolina: p = 0.0000
Normality test for Washington: p = 0.0000
Normality test for Wisconsin: p = 0.0000
Normality test for Arizona: p = 0.0000
Normality test for Texas: p = 0.0000
Levene's test for equal variances: p = 0.6846

ANOVA Results:
F-statistic: 0.7335
P-value: 0.6226
